In [1]:
import re, collections
from collections import defaultdict
import random

In [2]:
vocab_size = 1000
SPACE_NORMALIZER = re.compile("\s+")

def line_split(line):
    line = SPACE_NORMALIZER.sub(" ", line)
    line = line.strip()
    words = line.split()
    words = [word + "Ġ" for word in words]
    return words

In [3]:
def compute_pair_freqs(splits, word_freqs):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [4]:
def merge_pair(a, b, splits, word_freqs):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [5]:
corpus = [
    "BPE is an effective method for segmenting words into subword units, thereby enhancing the model's ability to manage rare or out-of-vocabulary words.",
    "It is reasonable to expect that adapting BPE will result in a higher BLEU score compared to using words exclusively.",
    "Additionally, the BPE-dropout can contribute to improved model generalization, as it exposes the model to diverse subword representations and variations. "
]

In [6]:
word_freqs = defaultdict(int)
for text in corpus:
    words = line_split(text)
    for word in words:
        word_freqs[word] += 1

vocab = defaultdict(int)
for word in word_freqs.keys():
    for letter in word:
        vocab[letter] += word_freqs[word]

splits = {word: [c for c in word] for word in word_freqs.keys()}

merges = {}
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits, word_freqs)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    if max_freq is None:
        break
    splits = merge_pair(*best_pair, splits, word_freqs)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab[best_pair[0] + best_pair[1]] = max_freq

vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [7]:
vocab

[('Ġ', 62),
 ('e', 38),
 ('o', 32),
 ('t', 31),
 ('i', 26),
 ('a', 24),
 ('r', 24),
 ('s', 22),
 ('n', 22),
 ('d', 17),
 ('l', 14),
 ('u', 10),
 ('eĠ', 10),
 ('c', 9),
 ('h', 9),
 ('m', 8),
 ('g', 8),
 ('sĠ', 8),
 ('or', 8),
 ('b', 7),
 ('p', 7),
 ('ti', 7),
 ('v', 6),
 ('w', 6),
 ('th', 6),
 ('dĠ', 6),
 ('to', 6),
 ('toĠ', 6),
 ('re', 6),
 ('tĠ', 6),
 ('on', 6),
 ('y', 5),
 ('an', 5),
 ('wor', 5),
 ('B', 4),
 ('E', 4),
 ('f', 4),
 ('en', 4),
 ('ng', 4),
 ('ngĠ', 4),
 ('el', 4),
 ('tion', 4),
 ('P', 3),
 (',', 3),
 ('-', 3),
 ('.', 3),
 ('x', 3),
 ('BP', 3),
 ('BPE', 3),
 ('word', 3),
 ('su', 3),
 (',Ġ', 3),
 ('yĠ', 3),
 ('theĠ', 3),
 ('mo', 3),
 ('mod', 3),
 ('model', 3),
 ('ab', 3),
 ('ar', 3),
 ('.Ġ', 3),
 ('ex', 3),
 ('er', 3),
 ('ation', 3),
 ('BPEĠ', 2),
 ('isĠ', 2),
 ('anĠ', 2),
 ('ec', 2),
 ('orĠ', 2),
 ('se', 2),
 ('tingĠ', 2),
 ('wordsĠ', 2),
 ('in', 2),
 ('sub', 2),
 ('subwor', 2),
 ('subwordĠ', 2),
 ('it', 2),
 ('ingĠ', 2),
 ('il', 2),
 ('ou', 2),
 ('s.Ġ', 2),
 ('exp', 2),


In [8]:
def tokenize(text):
    words = line_split(text)
    splits = [[c for c in word] for word in words]
    print(splits)
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    if random.random() < dropout:
                        i += 1
                        continue
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [9]:
dropout = 0
tokenize("BPE will result in a higher BLEU score.")

[['B', 'P', 'E', 'Ġ'], ['w', 'i', 'l', 'l', 'Ġ'], ['r', 'e', 's', 'u', 'l', 't', 'Ġ'], ['i', 'n', 'Ġ'], ['a', 'Ġ'], ['h', 'i', 'g', 'h', 'e', 'r', 'Ġ'], ['B', 'L', 'E', 'U', 'Ġ'], ['s', 'c', 'o', 'r', 'e', '.', 'Ġ']]


['BPEĠ',
 'willĠ',
 'resultĠ',
 'inĠ',
 'aĠ',
 'higherĠ',
 'BLEUĠ',
 'scor',
 'e',
 '.Ġ']

In [10]:
dropout = 0.05
tokenize("BPE will result in a higher BLEU score.")

[['B', 'P', 'E', 'Ġ'], ['w', 'i', 'l', 'l', 'Ġ'], ['r', 'e', 's', 'u', 'l', 't', 'Ġ'], ['i', 'n', 'Ġ'], ['a', 'Ġ'], ['h', 'i', 'g', 'h', 'e', 'r', 'Ġ'], ['B', 'L', 'E', 'U', 'Ġ'], ['s', 'c', 'o', 'r', 'e', '.', 'Ġ']]


['BPEĠ',
 'willĠ',
 'resultĠ',
 'inĠ',
 'aĠ',
 'higherĠ',
 'BLEUĠ',
 's',
 'c',
 'or',
 'e',
 '.Ġ']

In [11]:
# save merges
with open("merges.txt", "w") as f:
    for pair, merge in merges.items():
        f.write(f"{pair[0]} {pair[1]} -> {merge}\n")
        

In [12]:
new_merges = {}
# load merges
with open("merges.txt", "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        pair, merge = line.split(" -> ")
        pair = tuple(pair.split())
        new_merges[pair] = merge
new_merges

{('e', 'Ġ'): 'eĠ',
 ('s', 'Ġ'): 'sĠ',
 ('o', 'r'): 'or',
 ('t', 'i'): 'ti',
 ('t', 'h'): 'th',
 ('d', 'Ġ'): 'dĠ',
 ('t', 'o'): 'to',
 ('to', 'Ġ'): 'toĠ',
 ('r', 'e'): 're',
 ('t', 'Ġ'): 'tĠ',
 ('o', 'n'): 'on',
 ('a', 'n'): 'an',
 ('w', 'or'): 'wor',
 ('e', 'n'): 'en',
 ('n', 'g'): 'ng',
 ('ng', 'Ġ'): 'ngĠ',
 ('e', 'l'): 'el',
 ('ti', 'on'): 'tion',
 ('B', 'P'): 'BP',
 ('BP', 'E'): 'BPE',
 ('wor', 'd'): 'word',
 ('s', 'u'): 'su',
 (',', 'Ġ'): ',Ġ',
 ('y', 'Ġ'): 'yĠ',
 ('th', 'eĠ'): 'theĠ',
 ('m', 'o'): 'mo',
 ('mo', 'd'): 'mod',
 ('mod', 'el'): 'model',
 ('a', 'b'): 'ab',
 ('a', 'r'): 'ar',
 ('.', 'Ġ'): '.Ġ',
 ('e', 'x'): 'ex',
 ('e', 'r'): 'er',
 ('a', 'tion'): 'ation',
 ('BPE', 'Ġ'): 'BPEĠ',
 ('i', 'sĠ'): 'isĠ',
 ('an', 'Ġ'): 'anĠ',
 ('e', 'c'): 'ec',
 ('or', 'Ġ'): 'orĠ',
 ('s', 'e'): 'se',
 ('ti', 'ngĠ'): 'tingĠ',
 ('word', 'sĠ'): 'wordsĠ',
 ('i', 'n'): 'in',
 ('su', 'b'): 'sub',
 ('sub', 'wor'): 'subwor',
 ('subwor', 'dĠ'): 'subwordĠ',
 ('i', 't'): 'it',
 ('i', 'ngĠ'): 'ingĠ',
 ('i